In [7]:
from collections import namedtuple, defaultdict
import numpy as np
from enum import Enum


class DataLine:
    def __init__(self,
                 page_number: str = None,  #:Nстраницы
                 line_number: str = None,  #:Nстроки, сквозной
                 first_symbol: str = None,  #:Nсимвола, начального, сквозной
                 int_from_prev: str = None,  #:дельта-v-coord-up
                 int_from_next: str = None,  #:дельта-v-coord-down
                 h_left: str = None,  #:h-coord начала
                 h_right: str = None,  #:h-coord конца + 1
                 v: str = None,  #:v-coord
                 font0_size: str = None,  #:font-height0
                 font0_name: str = '',  #:font-name0
                 font1_size: str = None,  #:font-height1
                 font1_name: str = '',  #:font-name1
                 text: str = None,  #:TEXT
                 ):
        self.page_number = int(page_number) if len(page_number) else 0
        self.line_number = int(line_number) if len(line_number) else 0
        self.first_symbol = int(first_symbol) if len(first_symbol) else 0

        self.int_from_prev = int(int_from_prev) if len(int_from_prev) else 0
        self.int_from_next = int(int_from_next) if len(int_from_next) else 0

        self.h_left = int(h_left) if len(h_left) else 0
        self.h_right = int(h_right) if len(h_right) else 0
        self.v = int(v) if len(v) else 0

        self.font0_size = int(font0_size) if len(font0_size) else 0
        self.font0_name = font0_name
        self.font1_size = int(font1_size) if len(font1_size) else 0
        self.font1_name = font1_name

        self.text = text

        self.label = '?'

class State(Enum):
    NEUTRAL = 0
    PARAGRAPH = 1
    BUKVITSA = 2

class Label(Enum):
    PAR_STARTS = 'A'
    PAR_CONTINUES = 'B'
    HEADER = 'C'
    BUKVITSA = 'D'
    LIST = 'E'
    OTHER = '?'

In [16]:
import io
import os
from reportlab.pdfgen import canvas
from PyPDF2 import PdfFileWriter, PdfFileReader
import sys


def main(journal_name, paper_name):
    # чтение строк из txt-файла
    input_file_name = f'originals/{journal_name}/{paper_name}'
    data = []

    with open(os.path.join(os.path.dirname(os.path.curdir), input_file_name + '.txt'), "r", encoding="utf8") as read_file:
        for line in read_file:
            splitted = line.replace('\n', '').split(':')
            # print(splitted, "LEN", len(splitted))

            if len(splitted) > 13:
                rt = ':'.join(splitted[12:])
                data_line = DataLine(*splitted[:12], rt)
            elif len(splitted) == 13:
                data_line = DataLine(*splitted)
            else:
                pass
                assert len(splitted) >= 12, 'must have at least 12 elements'
            data.append(data_line)

    # границы абзацев
    margin_lefts = np.array([x.h_left for x in data])

    unique_margin_lefts, pos_margin_lefts = np.unique(margin_lefts, return_inverse=True)
    counts_margin_lefts = np.bincount(pos_margin_lefts)
    sorted_margin_lefts = np.argsort(counts_margin_lefts)[::-1]

    margin_rights = np.array([x.h_right for x in data])

    unique_margin_rights, pos_margin_rights = np.unique(margin_rights, return_inverse=True)
    counts_margin_rights = np.bincount(pos_margin_rights)
    sorted_margin_rights = np.argsort(counts_margin_rights)[::-1]

    EPS = 4
    EPS_FONT = 2

    left_keys = list()
    left_indents = defaultdict(int)
    for a, b in zip(unique_margin_lefts[sorted_margin_lefts[:15]], counts_margin_lefts[sorted_margin_lefts[:15]]):
        index = -1
        # print(a, b)
        for l in range(a - EPS, a + EPS + 1):
            if l in left_indents:
                index = l
        if index != -1:
            left_indents[index] += b
        else:
            left_indents[a] += b
            left_keys.append(a)

    left_len = 1
    left_sum = left_indents[left_keys[0]]
    while True:
        if left_indents[left_keys[left_len]] * 1.1 >= left_indents[left_keys[left_len - 1]]:
            left_sum += left_indents[left_keys[left_len]]
            left_len += 1
        else:
            break

    print(left_len)

    right_keys = list()
    right_indents = defaultdict(int)
    for a, b in zip(unique_margin_rights[sorted_margin_rights[:15]], counts_margin_rights[sorted_margin_rights[:15]]):
        index = -1
        # print(a, b)
        for l in range(a - EPS, a + EPS + 1):
            if l in right_indents:
                index = l
        if index != -1:
            right_indents[index] += b
        else:
            right_indents[a] += b
            right_keys.append(a)

    right_len = 1
    right_sum = right_indents[right_keys[0]]
    while True:
        if right_indents[right_keys[right_len]] * 1.1 >= right_indents[right_keys[right_len - 1]]:
            right_sum += right_indents[right_keys[left_len]]
            right_len += 1
        else:
            break

    print(right_len)

    if left_len != right_len:
        print(f'Error for detecting columns borders for {input_file_name}')
        return

    LEFT_BORDERS = np.sort(left_keys[:left_len])
    RIGHT_BORDERS = np.sort(right_keys[:right_len])

    # regular font

    font_names = defaultdict(int)

    REGULAR_FONT_NAME = None
    REGULAR_FONT_SIZE = None
    max_font_name = 0
    MAX_FONT_SIZE = 0
    MAX_FONT_COUNT = 0

    for x in data:
        if x.font0_size > MAX_FONT_SIZE:
            MAX_FONT_SIZE = x.font0_size
            MAX_FONT_COUNT = len(x.text)
        elif x.font0_size == MAX_FONT_SIZE:
            MAX_FONT_COUNT += len(x.text)

        if min(abs(x.h_left - LEFT_BORDERS)) <= EPS and min(abs(x.h_right - RIGHT_BORDERS)) <= EPS:
            # чтобы уж наверняка, выбираем только полные строки
            font_names[(x.font0_name, x.font0_size)] += 1
            font_names[(x.font1_name, x.font1_size)] += 1
            if font_names[(x.font0_name, x.font0_size)] > max_font_name:
                max_font_name = font_names[(x.font0_name, x.font0_size)]
                REGULAR_FONT_NAME = x.font0_name
                REGULAR_FONT_SIZE = x.font0_size
            if font_names[(x.font1_name, x.font1_size)] > max_font_name:
                max_font_name = font_names[(x.font1_name, x.font1_size)]
                REGULAR_FONT_NAME = x.font1_name
                REGULAR_FONT_SIZE = x.font1_size

    # bukvitsa
    IS_BUKVITSA = (MAX_FONT_COUNT == 1)
    
    
    
    def process_line(line, cur_state):
        # ищем колонку, куда помещается строчка
        ind_block = -1
        for i in range(len(LEFT_BORDERS)):
            l = LEFT_BORDERS[i]
            r = RIGHT_BORDERS[i]
            if l - EPS <= line.h_left and line.h_right <= r + EPS:
                ind_block = i

        # если колонка не обнаружена, выходим
        if ind_block == -1:
            return Label["OTHER"], State["NEUTRAL"]

        # иначе колонка найдена

        is_font0_same = abs(line.font0_size - REGULAR_FONT_SIZE) <= EPS_FONT
        is_font1_same = abs(line.font1_size - REGULAR_FONT_SIZE) <= EPS_FONT

        is_font0_regular = line.font0_name == REGULAR_FONT_NAME
        is_font1_regular = line.font1_name == REGULAR_FONT_NAME

        is_aligned_left = abs(LEFT_BORDERS[ind_block] - line.h_left) <= EPS
        is_aligned_right = abs(RIGHT_BORDERS[ind_block] - line.h_right) <= EPS

        bukvitsa_size = MAX_FONT_SIZE if IS_BUKVITSA else MAX_FONT_SIZE + 3 * EPS

        if is_aligned_left and is_aligned_right:
            if is_font0_same or is_font1_same:
                return Label["PAR_CONTINUES"], State["PARAGRAPH"]
            else:
                return Label["OTHER"], State["NEUTRAL"]
        elif is_aligned_left:
            # конец абзаца
            if is_font0_same or is_font1_same:
                return Label["PAR_CONTINUES"], State["PARAGRAPH"]
            # буквица
            elif abs(line.font0_size - bukvitsa_size) <= EPS or abs(line.font1_size - bukvitsa_size) <= EPS:
                return Label["BUKVITSA"], State["BUKVITSA"]
            # заголовок
            else:
                if (not is_font0_regular) or (not is_font1_regular):
                    return Label["HEADER"], State["NEUTRAL"]
                else:
                    return Label["OTHER"], State["NEUTRAL"]
        elif is_aligned_right:
            if cur_state == State["BUKVITSA"]:
                return Label["PAR_CONTINUES"], State["PARAGRAPH"]
            elif is_font0_same or is_font1_same:
                tokens = line.text.split()
                if len(tokens) > 0:
                    first_token = tokens[0]
                    filtered = ''.join(list(filter(lambda x: x in 'ЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ', first_token)))
                    if len(filtered) > 0:
                        return Label["PAR_STARTS"], State["PARAGRAPH"]
                    else:
                        return Label["LIST"], State["PARAGRAPH"]
                else:
                    return Label["OTHER"], State["NEUTRAL"]
            # заголовок
            else:
                return Label["HEADER"], State["NEUTRAL"]
        else:
            if (not is_font0_regular) and (not is_font1_regular):
                return Label["HEADER"], State["NEUTRAL"]
            else:
                tokens = line.text.split()
                if len(tokens) > 0:
                    first_token = tokens[0]
                    filtered = ''.join(list(filter(lambda x: x in 'ЁЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЯЧСМИТЬБЮ', first_token)))
                    if len(filtered) > 0:
                        return Label["OTHER"], State["NEUTRAL"]
                    else:
                        return Label["LIST"], State["PARAGRAPH"]
                else:
                    return Label["OTHER"], State["NEUTRAL"]
    
    
    
    
    output_file_name_txt = f'processed/{journal_name}/{paper_name}_l.txt'
    with open(os.path.join(os.path.dirname(os.path.curdir), output_file_name_txt), 'w', encoding='utf8') as output:
        cur_state = State["NEUTRAL"]

        PAR_INDENTS = defaultdict(int)

        for line in data:
            label, cur_state = process_line(line, cur_state)
            line.label = label.value
            if label == Label["PAR_STARTS"]:
                index = line.h_left
                for l in range(line.h_left - EPS, line.h_left + EPS + 1):
                    if l in PAR_INDENTS:
                        index = l
                PAR_INDENTS[index] += 1

        for line in data:
            if line.label == Label["OTHER"].value:
                # print('YES', line.text)
                index = line.h_left
                for l in range(line.h_left - EPS, line.h_left + EPS + 1):
                    if l in PAR_INDENTS:
                        index = l
                # print(index)
                if PAR_INDENTS[index] > 1:
                    line.label = Label["PAR_STARTS"].value

        for line in data:
            line_export = [line.page_number, line.line_number, line.first_symbol, line.int_from_prev,
                           line.int_from_next, line.h_left, line.h_right, line.v,
                           line.font0_size, line.font0_name, line.font1_size, line.font1_name,
                           line.label, line.text]

            line_export = ':'.join([str(y) for y in line_export])
            output.write(f'{line_export}\n')

    output_file_name_pdf = f'processed/{journal_name}/{paper_name}_l.pdf'

    color_dict = {'A': (1.0, 0.0, 0.0),
                  'B': (0.0, 0.1, 0.0),
                  'C': (0.0, 0.0, 1.0),
                  'D': (1.0, 0.0, 1.0),
                  'E': (1.0, 1.0, 0.0),
                  '?': (0.5, 0.5, 0.5),
                  }
    input_file = PdfFileReader(open(os.path.join(os.path.dirname(os.path.curdir), input_file_name + '.pdf'), "rb"))
    output_file = PdfFileWriter()
    cur_page = 0
    input_page = input_file.getPage(cur_page)

    c = canvas.Canvas('watermark.pdf')
    for line in data:

        if line.page_number - 1 == cur_page:
            # new page == old page
            c.setStrokeColorRGB(*color_dict[line.label])
            c.rect(line.h_left, line.v, line.h_right - line.h_left, 8, stroke=1, fill=0)

            # input_page.mergePage(watermark.getPage(0))
        else:
            c.save()
            watermark = PdfFileReader(open("watermark.pdf", "rb"))
            input_page.mergePage(watermark.getPage(0))
            output_file.addPage(input_page)

            cur_page = line.page_number - 1
            input_page = input_file.getPage(cur_page)

            c = canvas.Canvas('watermark.pdf')
            c.setStrokeColorRGB(*color_dict[line.label])
            c.rect(line.h_left, line.v, line.h_right - line.h_left, 8, stroke=1, fill=0)

    c.save()
    watermark = PdfFileReader(open("watermark.pdf", "rb"))
    input_page.mergePage(watermark.getPage(0))
    output_file.addPage(input_page)

    with open(os.path.join(os.curdir, output_file_name_pdf), "wb") as o:
        output_file.write(o)

In [9]:
main('aerospace', 'y:2015:i:1:p:1-17')

1
1


In [17]:
try:
    os.mkdir('processed')
except:
    pass

In [21]:
for journal in os.listdir('originals'):
    print(journal)
    try:
        os.mkdir(f'processed/{journal}')
    except:
        pass
    for paper in os.listdir(f'originals/{journal}'):
        if paper.endswith('.txt'):
            print(paper)
            paper_name = paper[:-4]
            try:
                main(journal, paper_name)
            except:
                pass

bulletin
917.txt
1
4
Error for detecting columns borders for originals/bulletin/917
acmeology
928.txt
2
2
biotechnology
y:2013:i:1:p:3-7.txt
biopreparations
y:2015:i:1:p:4-14.txt
1
2
Error for detecting columns borders for originals/biopreparations/y:2015:i:1:p:4-14
aerospace
y:2015:i:1:p:1-17.txt
1
1
abgb
y:2015:i:1:p:5-19.txt
anatomy
378.txt
1
1
aaresearch
y:2016:i:1:p:5-18.txt
1
2
Error for detecting columns borders for originals/aaresearch/y:2016:i:1:p:5-18
accounting
y:2016:i:2:p:48-53.txt
bricslaw
y:2014:i:1:p:4.txt
1
1
almclinmed
45.txt
3
1
Error for detecting columns borders for originals/almclinmed/45
adi-madi
y:1:i:1:p:2014.txt
a-surgeon
y:2015:i:2:p:3-7.txt
agx
y:2012:i:4:p:4-12.txt
1
4
Error for detecting columns borders for originals/agx/y:2012:i:4:p:4-12
akusherstvo
62.txt
actabiomedica
1281.txt
avia
1092.txt
archaeology
y:2013:i:1:p:2-27.txt
aterotromboz
y:2013:i:1:p:2-8.txt
1
1
bioterapevt
y:2015:i:1:p:3-10.txt
bibliotekovedenie
y:2011:i:2:p:1.txt
1
1
